In [1]:
import sqlite3
import os
import re

# Path to the folder containing your txt files
folder_path = 'D:/Athletic Screen 2.0/Output Files/'
db_path = 'D:/Athletic Screen 2.0/Output Files/movement_database_v2.db'

# Delete the database file if it exists to start fresh
if os.path.exists(db_path):
    os.remove(db_path)
    print(f"Deleted existing database at {db_path}")

# Connect to the SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Define the corrected table schemas for each movement
table_schemas = {
    'CMJ': '''CREATE TABLE IF NOT EXISTS CMJ (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                date TEXT,
                trial_name TEXT,
                JH_IN REAL,
                Peak_Power REAL,
                PP_FORCEPLATE REAL,
                Force_at_PP REAL,
                Vel_at_PP REAL,
                PP_W_per_kg REAL
              )''',

    'DJ':  '''CREATE TABLE IF NOT EXISTS DJ (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                date TEXT,
                trial_name TEXT,
                JH_IN REAL,
                Peak_Power REAL,
                PP_FORCEPLATE REAL,
                Force_at_PP REAL,
                Vel_at_PP REAL,
                PP_W_per_kg REAL,
                CT REAL,
                RSI REAL
              )''',

    'SLV': '''CREATE TABLE IF NOT EXISTS SLV (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                date TEXT, 
                trial_name TEXT,
                side TEXT,
                JH_IN REAL,
                PP_FORCEPLATE REAL,
                Force_at_PP REAL,
                Vel_at_PP REAL,
                PP_W_per_kg REAL
              )''',
    'NMT': '''CREATE TABLE IF NOT EXISTS NMT (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                name TEXT,
                date TEXT, 
                trial_name TEXT,
                NUM_TAPS_10s REAL,
                NUM_TAPS_20s REAL,
                NUM_TAPS_30s REAL,
                NUM_TAPS REAL
              )'''
}

# Create the tables in the database (if they don't exist)
for schema in table_schemas.values():
    cursor.execute(schema)

# Function to extract the client's name from the first line of the file
def extract_name(line):
    match = re.search(r'Data\\(.*?)[_\\]', line)
    if match:
        return match.group(1)
    return None

def extract_date(line):
    """
    Looks for a segment like 2025-05-22_ in the first-line path returned by Cortex.
    Returns '2025-05-22' or None if not found.
    """
    m = re.search(r'\\(\d{4}-\d{2}-\d{2})_', line)
    return m.group(1) if m else None
# ─────────────────────────────────────────────────────────────────────────────

# Function to insert data into the appropriate table
def insert_data_into_table(table_name, name, trial_name, variables):
    """
    `variables` still contains the leading dummy “1”.
    After we drop that each txt gives:
        CMJ : 8 numbers
        DJ  : 8 numbers
        SLV : 6 numbers
        NMT : 4 numbers
    We pick only the columns we store.
    """
    v = variables[1:]          # drop the leading “1”

    if table_name == 'CMJ':
        # keep indices 0,1,4,5,6,7  (→ six values)
        vals = [v[i] for i in (0, 1, 4, 5, 6, 7)]
        cursor.execute(
            """INSERT INTO CMJ
               (name, date, trial_name,
                JH_IN, Peak_Power,
                PP_FORCEPLATE, Force_at_PP, Vel_at_PP,
                PP_W_per_kg)
               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            (name, date, trial_name, *vals)
        )

    elif table_name == 'DJ':
        # keep every value (8 numbers)
        cursor.execute(
            """INSERT INTO DJ
               (name, date, trial_name,
                JH_IN, Peak_Power,
                PP_FORCEPLATE, Force_at_PP, Vel_at_PP,
                CT, RSI, PP_W_per_kg)
               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            (name, date, trial_name, *v)        # 8 numbers + 2 strings = 10
        )

    elif table_name == 'SLV':
        side = 'Left' if 'SLVL' in trial_name else 'Right'
        # keep indices 0,2,3,4,5  (→ five values)
        vals = [v[i] for i in (0, 2, 3, 4, 5)]
        cursor.execute(
            """INSERT INTO SLV
               (name, date, trial_name, side,
                JH_IN,
                PP_FORCEPLATE, Force_at_PP, Vel_at_PP,
                PP_W_per_kg)
               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)""",
            (name, date, trial_name, side, *vals)
        )

    elif table_name == 'NMT':
        cursor.execute(
            """INSERT INTO NMT
               (name, date, trial_name,
                NUM_TAPS_10s, NUM_TAPS_20s, NUM_TAPS_30s, NUM_TAPS)
               VALUES (?, ?, ?, ?, ?, ?, ?)""",
            (name, date, trial_name, *v)
        )
# Loop through the txt files in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith('.txt'):
        trial_name = os.path.splitext(file_name)[0]
        
        # Determine which table the file belongs to
        if 'CMJ' in trial_name:
            table_name = 'CMJ'
        elif 'DJ' in trial_name:
            table_name = 'DJ'
        elif 'SLVL' in trial_name or 'SLVR' in trial_name:
            table_name = 'SLV'
        elif 'NMT' in trial_name:
            table_name = 'NMT'
        else:
            continue  # Skip any files that don't match the naming pattern

        # Load the data from the txt file
        file_path = os.path.join(folder_path, file_name)
        try:
            with open(file_path, 'r') as f:
                # Extract the name from the first line
                first_line = f.readline().strip()
                name = extract_name(first_line)
                date = extract_date(first_line)

                # Print the extracted name to verify
                print(f"File: {file_name}, Extracted Name: {name}")

                if not name:
                    print(f"Name extraction failed for {file_name}, skipping.")
                    continue

                # Read all lines until we find the line with the actual numeric data
                # --- replace the old for-loop (line_num, line) with this: -------------------
                for line in f:
                    line = line.strip()
                    if not line:
                        continue
                
                    if re.match(r'^[-+]?\d', line):          # first real numeric row
                        variables = [float(v) for v in line.split()]
                        print(f"Processing file: {file_name}, Variables: {variables}")
                        insert_data_into_table(table_name, name, trial_name, variables)
                        break


        except Exception as e:
            print(f"Unexpected error with file {file_name}: {e}")

# Commit the changes and close the connection
conn.commit()
conn.close()

print("Data successfully inserted into the database.")


Deleted existing database at D:/Athletic Screen 2.0/Output Files/movement_database_v2.db
File: CMJ1.txt, Extracted Name: Ashton Seal
Processing file: CMJ1.txt, Variables: [1.0, 16.2, 513.0, 1.64, 314.0, 356.2, 1684.44, 211.44, 4.45]
File: SLVL1.txt, Extracted Name: Ashton Seal
Processing file: SLVL1.txt, Variables: [1.0, 9.5, 6194.0, 253.2, 1613.1, 156.9, 3.16]
File: CMJ2.txt, Extracted Name: Ashton Seal
Processing file: CMJ2.txt, Variables: [1.0, 15.1, 364.0, 1.36, 267.0, 372.4, 1665.38, 223.64, 4.66]
File: CMJ3.txt, Extracted Name: Ashton Seal
Processing file: CMJ3.txt, Variables: [1.0, 16.2, 513.0, 1.64, 314.0, 356.2, 1684.44, 211.44, 4.45]
File: DJ1.txt, Extracted Name: Ashton Seal
Processing file: DJ1.txt, Variables: [1.0, 19.6, 1462.0, 642.8, 1736.54, 370.16, 0.6, 1.66, 8.03]
File: DJ2.txt, Extracted Name: Ashton Seal
Processing file: DJ2.txt, Variables: [1.0, 20.6, 1720.0, 622.2, 1707.4, 364.4, 0.56, 1.86, 7.78]
File: DJ3.txt, Extracted Name: Ashton Seal
Processing file: DJ3.txt

In [2]:
"""
Create a longitudinal “comparison” report for a single athlete.

• Reads every assessment for that athlete from Athletic_Screen_All_data_v2.db
• Builds one small line-chart per metric (dates on x-axis, metric on y-axis)
• Keeps your existing scatter-plots (Force@PP vs Vel@PP) – but now draws **all**
  dots, one per assessment day
• Calculates %-change between the last two assessments and writes it under
  each chart
• Saves the report with a unique, date-stamped filename (won’t overwrite)
-------------------------------------------------------------------------------
Dependencies:
  pip install matplotlib pandas scipy python-docx docx2txt pillow
"""

import os
import sqlite3
from datetime import date
import tempfile
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from docx import Document
from docx.shared import Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH

# --------------------------------------------------------------------------- #
# -------------------------  CONFIGURABLE PATHS  ---------------------------- #
# --------------------------------------------------------------------------- #
DB_PATH      = r'D:\Athletic Screen 2.0\Output Files\Athletic_Screen_All_data_v2.db'
REPORTS_DIR  = r'G:\My Drive\Athletic Screen 2.0 Reports\Comparison Reports'
LOGO_PATH    = r'8ctane Baseball - Black abd Blue BG.jpeg'      # adjust if needed
IMG_DIR_NAME = "Images"                                         # sub-folder for pngs

# ---- fetch the only athlete in the temp DB -------------------------------
temp_db = r'D:\Athletic Screen 2.0\Output Files\movement_database_v2.db'
with sqlite3.connect(temp_db) as tmp_conn:
    tmp_cur = tmp_conn.cursor()
    name_row = tmp_cur.execute("SELECT DISTINCT name FROM CMJ").fetchone()
    if not name_row:
        raise SystemExit("No athlete found in movement_database_v2.db")
    CLIENT_NAME = name_row[0]


# ------------- open DB ------------------------------------------------------ #
conn   = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# quick sanity-check – is the athlete in CMJ at all?
cursor.execute("SELECT COUNT(*) FROM CMJ WHERE name = ?", (CLIENT_NAME,))
if cursor.fetchone()[0] == 0:
    raise SystemExit(f"No rows for '{CLIENT_NAME}' in the All-data DB.")

# --------------------------------------------------------------------------- #
#                HELPER – fetch each table into a tidy DataFrame              #
# --------------------------------------------------------------------------- #
TABLE_METRICS = {
    "CMJ": ["JH_IN", "PP_FORCEPLATE", "PP_W_per_kg",
            "Force_at_PP", "Vel_at_PP"],
    "DJ" : ["JH_IN", "PP_FORCEPLATE", "PP_W_per_kg",
            "Force_at_PP", "Vel_at_PP", "CT", "RSI"],
    "SLV": ["side",  # keep side to split later
            "JH_IN", "PP_FORCEPLATE", "PP_W_per_kg",
            "Force_at_PP", "Vel_at_PP"],
    "NMT": ["NUM_TAPS_10s"],
}

def load_table(table: str) -> pd.DataFrame:
    cols = ["date"] + TABLE_METRICS[table]
    sql  = f"SELECT {', '.join(cols)} FROM {table} WHERE name = ? ORDER BY date"
    df   = pd.read_sql_query(sql, conn, params=(CLIENT_NAME,))
    df['date'] = pd.to_datetime(df['date'])
    return df

# --------------------------------------------------------------------------- #
#                         PLOTTING HELPERS                                    #
# --------------------------------------------------------------------------- #
plt.rcParams.update({
    "figure.facecolor": "#181818",
    "axes.facecolor"  : "#303030",
    "axes.edgecolor"  : "white",
    "axes.labelcolor" : "slategrey",
    "xtick.color"     : "lightgrey",
    "ytick.color"     : "lightgrey",
    "grid.color"      : "dimgrey",
    "text.color"      : "white",
})

def scatter_trend(df: pd.DataFrame, metric: str, table: str, out_path: str):
    """
    • Plots every individual trial as grey dots
    • Plots the *mean of each assessment date* as large coloured dots
    • Draws a trend-line that connects ONLY the first-date mean ➝ last-date mean
    • Adds a text-box with:
        – mean for each date
        – %Δ (last vs first)
        – percentile of each mean vs all-athlete reference distribution
    """
    # ---- reference distribution for percentile ----------------------------
    ref_vals = pd.read_sql_query(
        f"SELECT {metric} FROM {table} WHERE {metric} IS NOT NULL",
        conn
    )[metric].astype(float).values

    by_day = (df.groupby('date')[metric]
                .mean()
                .reset_index()
                .sort_values('date'))
    # colours: first = blue, last = orange (mid dates = grey if present)
    colours = ['cornflowerblue']*len(by_day)
    colours[-1] = 'orange'

    fig, ax = plt.subplots()
    # raw trials
    ax.scatter(df['date'], df[metric], s=25, c='grey', alpha=.4)

    # date means
    ax.scatter(by_day['date'], by_day[metric],
               s=100, c=colours, edgecolors='black', zorder=3)

    # trend line first ➝ last
    ax.plot(by_day['date'].iloc[[0, -1]],
            by_day[metric].iloc[[0, -1]],
            c='white', ls='--', lw=2)

    # % change
    pct_change = (by_day[metric].iloc[-1] - by_day[metric].iloc[0]) \
                 / by_day[metric].iloc[0]
    pct_txt = f"%Δ: {pct_change:+.1%}"
    pct_col = "lime" if pct_change > 0 else "tomato"

    # text-box --------------------------------------------------------------
    lines = [pct_txt]
    for d, v in zip(by_day['date'], by_day[metric]):
        prc = stats.percentileofscore(ref_vals, v)
        lines.append(f"{d.date()}: {v:.2f}  ({prc:.0f}ᵗʰ)")
    txt = "\n".join(lines)

    ax.text(0.98, 0.02, txt,
            transform=ax.transAxes,
            va='bottom', ha='right',
            fontsize=9, color='white',
            bbox=dict(facecolor="#181818",
                      edgecolor="#404040", pad=5))

    ax.set_xlabel("Date")
    ax.set_ylabel(metric.replace('_', ' '))
    fig.autofmt_xdate()
    ax.grid(True)
    fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)


def scatter_force_vel(df: pd.DataFrame, force_col: str, vel_col: str,
                      table: str, title: str, out_path: str):
    """
    Reference dots (all athletes) in pale grey.
    Athlete dots coloured by date (earlier→later = dark→bright).
    """
    # reference cloud -------------------------------------------------------
    ref = pd.read_sql_query(
        f"SELECT {force_col}, {vel_col} FROM {table} "
        f"WHERE {force_col} IS NOT NULL AND {vel_col} IS NOT NULL",
        conn
    ).astype(float)

    fig, ax = plt.subplots(figsize=(4, 4))
    ax.scatter(ref[force_col], ref[vel_col],
               c='lightgrey', s=15, alpha=.4, label='Reference')

    # athlete dots ----------------------------------------------------------
    dates_ord = pd.to_datetime(df['date']).map(pd.Timestamp.toordinal)
    sc = ax.scatter(df[force_col], df[vel_col],
                    c=dates_ord, cmap='viridis',
                    s=80, edgecolors='black', label='Athlete')

    ax.set_xlabel(force_col.replace('_', ' '))
    ax.set_ylabel(vel_col.replace('_', ' '))
    ax.set_title(title, color='white', fontsize=10, pad=10)
    ax.grid(True)
    fig.colorbar(sc, ax=ax, label='Date (ordinal)')
    fig.savefig(out_path, bbox_inches="tight")
    plt.close(fig)

# --------------------------------------------------------------------------- #
#                     DOCX PREPARATION                                        #
# --------------------------------------------------------------------------- #
# build export paths (date-stamped, no overwrite)
latest_date = cursor.execute(
    "SELECT MAX(date) FROM CMJ WHERE name = ?", (CLIENT_NAME,)
).fetchone()[0] or date.today().strftime("%Y-%m-%d")

parts = CLIENT_NAME.split(', ')
client_name_rev = f"{parts[1]}_{parts[0]}" if len(parts)==2 else CLIENT_NAME
base_name = f"Comparison_Report_{client_name_rev}_{latest_date}"

reports_dir = os.path.abspath(REPORTS_DIR)
os.makedirs(reports_dir, exist_ok=True)

docx_path = os.path.join(reports_dir, base_name + ".docx")
img_root  = os.path.join(reports_dir, IMG_DIR_NAME, base_name)
os.makedirs(img_root, exist_ok=True)

i = 1
while os.path.exists(docx_path):
    docx_path = os.path.join(reports_dir, f"{base_name}_{i}.docx")
    img_root  = os.path.join(reports_dir, IMG_DIR_NAME, f"{base_name}_{i}")
    os.makedirs(img_root, exist_ok=True)
    i += 1

# --------------------------------------------------------------------------- #
#                        GENERATE THE REPORT                                  #
# --------------------------------------------------------------------------- #
doc = Document()
if os.path.exists(LOGO_PATH):
    doc.add_picture(LOGO_PATH, width=Inches(4.0))
    doc.paragraphs[-1].alignment = WD_ALIGN_PARAGRAPH.CENTER

doc.add_heading("Athlete Longitudinal Report", level=1)
doc.add_paragraph(f"Athlete:  {CLIENT_NAME}")
doc.add_paragraph(f"Generated: {date.today().strftime('%B %d, %Y')}")

with tempfile.TemporaryDirectory() as tmpdir:

    # -------------- CMJ ----------------------------------------------------- #
    df_cmj = load_table("CMJ")
    if not df_cmj.empty:
        doc.add_heading("CMJ", level=2)

        for m in ["JH_IN", "PP_FORCEPLATE", "PP_W_per_kg",
                  "Force_at_PP", "Vel_at_PP"]:
            plot_path = os.path.join(tmpdir, f"CMJ_{m}.png")
            scatter_trend(df_cmj, m, "CMJ", plot_path)
            doc.add_paragraph(m.replace('_',' '), style='Heading 3')
            doc.add_picture(plot_path, width=Inches(5.5))

        # scatter        -------------------
        scat = os.path.join(tmpdir, "CMJ_force_vel.png")
        scatter_force_vel(df_cmj, "Force_at_PP", "Vel_at_PP", "CMJ",
                          "CMJ Force vs Velocity", scat)
        doc.add_paragraph("Force vs Velocity", style='Heading 3')
        doc.add_picture(scat, width=Inches(5.5))

    # -------------- DJ ------------------------------------------------------ #
    df_dj = load_table("DJ")
    if not df_dj.empty:
        doc.add_page_break()
        doc.add_heading("DJ", level=2)

        for m in ["JH_IN", "PP_FORCEPLATE", "PP_W_per_kg",
                  "Force_at_PP", "Vel_at_PP", "CT", "RSI"]:
            if m not in df_dj.columns:   # some metrics optional
                continue
            plot_path = os.path.join(tmpdir, f"DJ_{m}.png")
            scatter_trend(df_dj, m, "DJ", plot_path)
            doc.add_paragraph(m.replace('_',' '), style='Heading 3')
            doc.add_picture(plot_path, width=Inches(5.5))

        scat = os.path.join(tmpdir, "DJ_force_vel.png")
        scatter_force_vel(df_dj, "Force_at_PP", "Vel_at_PP", "DJ",
                          "DJ Force vs Velocity", scat)
        doc.add_paragraph("Force vs Velocity", style='Heading 3')
        doc.add_picture(scat, width=Inches(5.5))

    # -------------- SLV (split sides) --------------------------------------- #
    df_slv = load_table("SLV")
    if not df_slv.empty:
        doc.add_page_break()
        doc.add_heading("Single-Leg Vertical (SLV)", level=2)

        for side in ["Left", "Right"]:
            sub = df_slv[df_slv['side']==side].copy()
            if sub.empty:
                continue
            doc.add_heading(f"{side} leg", level=3)
            for m in ["JH_IN", "PP_FORCEPLATE", "PP_W_per_kg",
                      "Force_at_PP", "Vel_at_PP"]:
                plot_path = os.path.join(tmpdir, f"SLV_{side}_{m}.png")
                scatter_trend(sub, m, "SLV", plot_path)
                doc.add_paragraph(m.replace('_',' '), style='Heading 4')
                doc.add_picture(plot_path, width=Inches(5.0))

        # scatter – one colour per date (left & right merged)
        scat = os.path.join(tmpdir, "SLV_force_vel.png")
        scatter_force_vel(df_slv, "Force_at_PP", "Vel_at_PP", "SLV",
                          "SLV Force vs Velocity", scat)
        doc.add_heading("Force vs Velocity (both legs)", level=3)
        doc.add_picture(scat, width=Inches(5.5))

    # -------------- NMT ----------------------------------------------------- #
    df_nmt = load_table("NMT")
    if not df_nmt.empty:
        doc.add_page_break()
        doc.add_heading("Neuromuscular Taps (NMT)", level=2)

        plot_path = os.path.join(tmpdir, "NMT_taps.png")
        scatter_trend(df_nmt, "NUM_TAPS_10s", "NMT", plot_path)
        doc.add_paragraph("Number of taps in 10 s", style='Heading 3')
        doc.add_picture(plot_path, width=Inches(5.5))
        
    # ---- SAVE DOCX before tmpdir disappears ----
    doc.save(docx_path)
    print(f"✅  Report saved to\n    {docx_path}")

conn.close()
print("Comparison Completed")

# Path to the folder containing ASCII .txt files
ascii_folder = r"D:/Athletic Screen 2.0/Output Files/"

# Remove all .txt files in the ascii_folder
for filename in os.listdir(ascii_folder):
    if filename.lower().endswith(".txt"):
        file_path = os.path.join(ascii_folder, filename)
        try:
            os.remove(file_path)
            print(f"Deleted: {file_path}")
        except Exception as e:
            print(f"Failed to delete {file_path}: {e}")

print("All ASCII .txt files cleared after ingestion.")


SystemExit: No rows for 'Ashton Seal' in the All-data DB.

C:\Users\q\PycharmProjects\Athletic Screen Reporting\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
